# Chapter 2
+ 아래로 내려가면서 구현
+ 다 구현되면 .py 파일로 모듈화

In [ ]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
'''
symbol = 'ES=F'
f = open("ALPHA_KEY", 'r')
key = f.readline()
f.close()
# AlphaVantage에서 데이터를 받아온다.
ts  = TimeSeries(key=key, output_format='pandas')
tmp, meta_tmp = ts.get_intraday(symbol=symbol,interval='1min', outputsize='full')
tmp.index = pd.to_datetime(tmp.index)
'''

# 틱 데이터
path = 'MiniKospi/'
# 파일명이 일관성이 있으며 시간순으로 오름차순이어야함. 
# 아니면 별도의 sort 또는 예외처리를 해야한다. 
list_file = os.listdir(path)

data = pd.DataFrame(columns=['close','volume']) 
data.index.names = ['timestamp']
cnt=0


# 일단은 전체 데이터를 하나의 dataframe이 넣어서 돌리자
# 메모리 이슈 생각하면 파일을 쪼개서 하나씩 처리하면 되지만 아직은 데이터 셋이 그렇게 크지 않음
# 다만 항상 분할해서 처리해야함을 염두해 두자.
for file in list_file :
    cnt+=1
    print('reading ' + path + file + ' ' + str(cnt) +'/' + str(len(list_file)) )
    tmp = pd.read_csv(path+file,index_col=0)
    tmp = tmp.drop(columns=['date','time','sell','buy'])
    # conver to datetime
    tmp.index = pd.to_datetime(tmp.index)
    tmp.index.names = ['timestamp']
    data = data.append(tmp)
    # display(tmp.head())
    # 일단 3개만.
    if cnt > 2 :
        break
# display
display(data.head())
display(data.tail())

In [ ]:
data['close'].plot()

## Exponential Weighted Moving Average

https://www.investopedia.com/terms/e/ema.asp


Calculating the EMA
To calculate an EMA, you must first compute the simple moving average (SMA) over a particular time period. The calculation for the SMA is straightforward: it is simply the sum of the stock's closing prices for the number of time periods in question, divided by that same number of periods. So, for example, a 20-day SMA is just the sum of the closing prices for the past 20 trading days, divided by 20.

Next, you must calculate the multiplier for smoothing (weighting) the EMA, which typically follows the formula: [2 ÷ (selected time period + 1)]. So, for a 20-day moving average, the multiplier would be [2/(20+1)]= 0.0952.

Finally, to calculate the current EMA, the following formula is used: [Closing price-EMA (previous day)] x multiplier + EMA (previous day)

내가 이해하기로는 EWMA 의 식은

$ \text{SMA}_t = \frac{\sum_{k=t-\text{n}}^t \text{value}_k}{\text{n}}$ 

$ \large \text{weight} = \frac{2}{\text{n} + 1}$

$ \text{EMA}_t = \text{SMA}_t * \text{weight} + EMA_{t-1}*(1-\text{weight}) $

인데


+ https://stackoverflow.com/questions/42869495/numpy-version-of-exponential-weighted-moving-average-equivalent-to-pandas-ewm/51392341#51392341
+ mlfinlab 에서 사용한 ewma 코드 출처

```python
   n = arr_in.shape[0]
    ewma = np.empty(n, dtype=float64)
    alpha = 2 / float(window + 1)
    w = 1
    ewma_old = arr_in[0]
    ewma[0] = ewma_old
    for i in range(1, n):
        w += (1-alpha)**i
        ewma_old = ewma_old*(1-alpha) + arr_in[i]
        ewma[i] = ewma_old / w
    return ewma
    
```

좀 다르다?   
```ewma_old = ewma_old*(1-alpha) + arr_in[i]   
ewma[i] = ewma_old / w```  
계산해보면 같을거 같긴한데, 일단 짜보자

In [29]:
def ewma_weight(window):
    return 2/(window+1)

## Unit of Bar

일단 주 단위로 구해보자

https://stackoverflow.com/questions/41625077/python-pandas-split-a-timeserie-per-month-or-week  

[Grouper](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html)

In [21]:
weekly = [g for n, g in data.groupby(pd.Grouper(level='timestamp', freq='W'))]

## Tick Imbalance Bars

### step 1  
$
 b_t  = 
 \begin{cases}
 \Delta b_{t-1}  &\text{if  \( \Delta P_t = 0\)}\\
 \frac{|\Delta P_t|}{\Delta P_t} &\text{if \( \Delta P_t \neq 0 \)}
 \end{cases}
$
### step 2  
$
   \theta_T = \sum_{t = 1}^T b_t
$
 
 
현재 주 단위로 ```weekly``` 리스트에 들어있음. weekly

일단 weekly[0] 를 거르고 - 월~금이 다 있는지 장담할 수 없음 - weekly[1] 을 기준 삼아 진행해 나가자. 

일단 weekly[1] 에서 $P[b_t = 1]$ 은 구할수 있는데 문제는 $E_0[T]$ 의 설정. 일단 구하고자 하는 것은

### step 3  
$\text{arg } \text{min} \{  |\theta_t| \geq E_0[T] \mid 2P[b_t = 1] -1 \mid \} $.

mlfinlab 에서는 

```exp_num_ticks_init=100000``` 로 해두었지만 이건 케바케가 굉장히 심한 요소.

$ E_0[T] \mid 2P[b_t = 1] -1 \mid $ 는  원래 $ E_0[T] \mid P[b_t = 1] -P[b_t = -1] \mid $ 이는
imbalance 를 측정하기 위함.  
그렇다면 t=0 를 기준으로 잡게 하는 $E_0[T]$ 를 구하면 되지 않을까? 

그러면 생각나는 것이..
+ $\theta_0$ 의 평균
+ $\theta_0$ 의 최댓값
+ $\theta_0$ 의 평균 + 표준 편차 : 이게 좋겠다.  

일단 $P_t$와 $\theta_t$를 구하는 부분 부터, 나중에 모듈화 해야하는 거 고려하면서 프로토타입 작성.

In [43]:
data.columns

Index(['close', 'volume'], dtype='object')

In [35]:
def for_init(data,threshold=500,value='close'):
    # target variables
    pr = 0
    e_0 =0
    # imtermediate variables
    theta = []
    
    cur_value = 0
    
    prev_p =0
    d_p =0
    cur_p = data.iloc[0]
    cur_p = cur_p[value]
    prev_b =0
    cur_b =0
    cum_b=0
    # p[b_t = 1]
    num_tick=0 
    num_p_1 =0 
    
    for i in range(len(data.index)):
        # tick
        cur_value += 1
        if(cur_value >= threshold):
            cur_tick = 0    
            # step 1
            prev_p = cur_p
            cur_p = i.iloc[i][value]
            d_p = cur_p - prev_p
            
            prev_b = cur_b
            if d_p != 0:
                cur_b = abs(d_p)/d_p                
            else :
                if prev_b == 0 : # 예외처리
                    cur_b = 1
                else :
                    cur_b = prev_b          
            # step 2        
            cum_b += cur_b
            theta.append(cum_b )
            num_tick+=1
            if(cur_b==1):
                num_p_1 +=1
    pr = num_p_1/num_tick
    e_0 = np.mean(theta) + np.std(theta)
    # display(theta)
    return e_0,pr

이제는 ewma 도 고려해서, tick bar 또한 생성

In [39]:
type(data.index[0])

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
def for_procedure(data,e_0,pr,threshold=500,value='close'):
    # target variables
    pr = 0
    e_0 =0
    # imtermediate variables
    theta = []
    
    cur_value = 0
    
    prev_p =0
    d_p =0
    cur_p = data.iloc[0]
    cur_p = cur_p[value]
    prev_b =0
    cur_b =0
    cum_b=0
    # p[b_t = 1]
    num_tick=0 
    num_p_1 =0 
    
    for i in data.values:
        # tick
        cur_value += 1
        if(cur_value >= threshold):
            cur_tick = 0    
            # step 1
            prev_p = cur_p
            cur_p = i[0]
            d_p = cur_p - prev_p
            
            prev_b = cur_b
            if d_p != 0:
                cur_b = abs(d_p)/d_p                
            else :
                if prev_b == 0 : # 예외처리
                    cur_b = 1
                else :
                    cur_b = prev_b     
            # step 2
            cum_b += cur_b
            theta.append(cum_b )
            num_tick+=1
            if(cur_b==1):
                num_p_1 +=1
    
    weight = ewma_weight(num_tick)
    pr = (num_p_1/num_tick)*weight + pr*(1-weight)
    # step 3
    tick = pd.DataFrame(columns=['close','volume']) 
    tick.index.names = ['timestamp']
    
    num_tick = 0
    for i in range(len(theta)) :
        num_tick+=1
        if(abs(theta[i]) > e_0 * abs(2*pr - 1)):
            tick.append(i)
    return e_0,pr,tick

In [36]:
threshold = 100


# init
e_0,pr =  for_init(weekly[1],threshold)  
print(str(e_0) + ' ' + str(pr))
# procedure
#for tmp in weekly[1:]: 
 #   e_0,pr,t_tick = for_procedure(data,e_0,pr,threshold=500,value='close')



AttributeError: 'int' object has no attribute 'loc'

구현하다 만것

```python
def tick_bar(data,threshold=100):
    num_tick=0
    c_tick = 0 # current tick counter
    prev_p =0
    cur_p =0
    d_P_t =0
    b_t=[]
    t_close=0
    for i in data.index:
            c_tick += 1
            num_tick +=1
            t_close += data.loc[i]['close']
            if(c_tick == threshold):
                '''
                    tick rule defines a sequence b_t
                '''
                prev_p = cur_p
                cur_p = t_close/threshold
                #cur_p = data.loc[i]['close']
                d_p = cur_p - prev_p
                if d_p != 0:
                    b_t.append(abs(d_p)/d_p )
                else :
                    b_t.append(b_t[-1])
                if(b_t[-1]==1):
                    num_buy+=1
                c_tick = 0
                t_close = 0
    sum_b = 0            
    theta_t = []
    for i in b_t : 
        sum_b += i
        theta_t.append(sum_b)
    '''
        Second, we compute the expected value of e_t at the beginning of the bar
    '''
            
    '''
        Third, we define a tick imbalane bar (TIB) as a T^*-contiguous subset of ticks
    '''
    
    return theta_t
```

In [ ]:
for i in [10,100,1000] :
    plt.figure()
    plt.title('tick_imbalance with threshold '+ str(i))
    plt.plot(tick_bar(data,i))

### Volume Bars

In [ ]:
def volume_bar(data,threshold=100):
    num_tick=0    
    c_tick = 0 # current tick counter
    prev_p =0
    cur_p =0
    d_P_t =0
    
    temp_v = 0
    v_t=[]
    
    b_t=[]
        
    num_buy = 0
    prob = 0
    
    for i in data.index:
            c_tick += 1
            temp_v +=data.loc[i]['volume']
            if(c_tick == threshold):
                num_tick +=1
                prev_p = cur_p
                cur_p = data.loc[i]['close']
                d_p = cur_p - prev_p
                if d_p != 0:
                    b_t.append(abs(d_p)/d_p )
                else :
                    b_t.append(b_t[-1])
                if(b_t[-1]==1):
                    num_buy+=1
                v_t.append(temp_v)    
                    
                prob = 0
                
                temp_v = 0                
                c_tick = 0

    sum_b = 0            
    theta_t = []
    for i in range(len(b_t)) : 
        sum_b += b_t[i]*v_t[i]
        theta_t.append(sum_b)
        
    E_theta_t = 0
    
    
    return theta_t

In [ ]:
for i in [10,100,1000] :
    plt.figure()
    plt.title('volume imbalance with threshold '+ str(i))
    plt.plot(volume_bar(data,i))


In [ ]:
data.head()